In [ ]:
# Author: Lennard Alms // Compare to boilerplate to see what parts are different

In [ ]:
# edit this when working in a local environment
!wget "https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip" --no-verbose
!unzip spatio_merged_data_iss.zip
!rm spatio_merged_data_iss.zip

2021-01-28 12:14:23 URL:https://storage.googleapis.com/laubenthal_spatiolab/spatio_merged_data_iss.zip [37282886/37282886] -> "spatio_merged_data_iss.zip" [1]
Archive:  spatio_merged_data_iss.zip
   creating: input/
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/grid_ML.geojson   
  inflating: __MACOSX/input/._grid_ML.geojson  
  inflating: input/internet_ML.csv   
  inflating: __MACOSX/input/._internet_ML.csv  
  inflating: input/satelite.png      
  inflating: __MACOSX/input/._satelite.png  
  inflating: input/weather.csv       
  inflating: __MACOSX/input/._weather.csv  
  inflating: input/social_pulse_ML.csv  
  inflating: __MACOSX/input/._social_pulse_ML.csv  


In [ ]:
# edit this when working in a local environment
!rm -rf functions
!git clone https://github.com/markuslaubenthal/lab_st.git functions

Cloning into 'functions'...
remote: Enumerating objects: 314, done.
remote: Total 314 (delta 0), reused 0 (delta 0), pack-reused 314
Receiving objects: 100% (314/314), 170.61 KiB | 425.00 KiB/s, done.
Resolving deltas: 100% (197/197), done.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, activations
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf

In [ ]:
from functions.preprocessing.DataImport import load_and_scale_internet, load_and_scale_satelite, load_and_scale_social, load_and_scale_weather, create_space_invariant
from functions.preprocessing.DataGeneration import generate_dataset, generate_label, getFileHandler, get_datasets_from_file
from functions.postprocessing.ErrorEvaluation import calculate_errors
from functions.preprocessing.TestTrainSplit import seven_days_train_test_split
from functions.model.HadamardLayer import HadamardLayer

In [ ]:
f = getFileHandler("training_data.h5")

In [ ]:
internet, internet_origin, internet_min, internet_max = load_and_scale_internet('input/internet_ML.csv', f, log10=True)
#satelite = load_and_scale_satelite('input/satelite.png', f)
#social = load_and_scale_social('input/social_pulse_ML.csv', f)
#weather = load_and_scale_weather('input/weather.csv', f)
#hour, weekday, holiday = create_space_invariant(f)

In [ ]:
steps_back = np.array([0, 1, 2, 3, 20, 21, 22, 23, 24, 143, 165, 166, 167]) + 1
steps_back = steps_back[::-1]
x = generate_dataset(internet, steps_back, 168, f, "x")
y = generate_label(internet, 168, f)

In [ ]:
x_train, x_test = seven_days_train_test_split(x, 168)
y_train, y_test = seven_days_train_test_split(y, 168)

In [ ]:
y = y[:].reshape(y.shape[0], 100, 100)
y_train = y_train[:].reshape(y_train.shape[0], 100, 100)
y_test = y_test[:].reshape(y_test.shape[0], 100, 100)

In [ ]:
xt = np.zeros((x.shape[0],x.shape[3],x.shape[2],x.shape[1]))
yt = np.zeros((x.shape[0],x.shape[2],x.shape[1]))
for t in range(x.shape[0]):
  xt[t] = x[t].T
  yt[t] = y[t].T

xt_train = np.zeros((x_train.shape[0],x_train.shape[3],x_train.shape[2],x_train.shape[1]))
yt_train = np.zeros((x_train.shape[0],x_train.shape[2],x_train.shape[1]))
for t in range(x_train.shape[0]):
  xt_train[t] = x_train[t].T
  yt_train[t] = y_train[t].T

xt_test = np.zeros((x_test.shape[0],x_test.shape[3],x_test.shape[2],x_test.shape[1]))
yt_test = np.zeros((x_test.shape[0],x_test.shape[2],x_test.shape[1]))
for t in range(x_test.shape[0]):
  xt_test[t] = x_test[t].T
  yt_test[t] = y_test[t].T

In [ ]:
xt = xt[:,:,:,:,np.newaxis]
xt_train = xt_train[:,:,:,:,np.newaxis]
xt_test = xt_test[:,:,:,:,np.newaxis]

In [ ]:
lay = [8]
filt = [4]
pat = [(3,3)]
for _l in lay:
  for _f in filt:
    for _p in pat:
      
      name = f'log:True, layers:{_l}, filters:{_f}, patch:{_p}'
      print(name)
      
      model = keras.models.Sequential()
      
      model.add(keras.Input(shape=(xt.shape[1],xt.shape[2],xt.shape[3])))

      model.add(layers.Reshape(((xt.shape[1],xt.shape[2],xt.shape[3],1))))

      for __l in range(_l):
        ret_seq = __l != _l - 1
        model.add(layers.ConvLSTM2D(_f, 
                                    _p, 
                                    padding='same', 
                                    return_sequences=ret_seq, 
                                    data_format='channels_last'))
      
      model.add(HadamardLayer(name='H1'))
      model.add(layers.Lambda(lambda x : K.sum(x, axis=3)))

      lr = 0.01
      epochs = 50
      model.compile(optimizer=keras.optimizers.Adam(lr=lr, decay= lr/float(epochs)), loss='mse')
      model.fit(xt_train, yt_train, validation_data=(xt_test,yt_test),epochs=epochs)

      predt = model.predict(xt)
      pred = np.zeros(predt.shape)
      for t in range(predt.shape[0]):
        pred[t] = predt[t].T

      calculate_errors(pred, internet_origin, internet_min, internet_max, log10=True)


log:True, layers:8, filters:4, patch:(3, 3)
Epoch 1/50
36/36 [==============================] - 82s 2s/step - loss: 0.1508 - val_loss: 0.0717
Epoch 2/50
36/36 [==============================] - 65s 2s/step - loss: 0.0365 - val_loss: 0.0329
Epoch 3/50
36/36 [==============================] - 65s 2s/step - loss: 0.0181 - val_loss: 0.0263
Epoch 4/50
36/36 [==============================] - 65s 2s/step - loss: 0.0153 - val_loss: 0.0130
Epoch 5/50
36/36 [==============================] - 65s 2s/step - loss: 0.0127 - val_loss: 0.0212
Epoch 6/50
36/36 [==============================] - 65s 2s/step - loss: 0.0116 - val_loss: 0.0106
Epoch 7/50
36/36 [==============================] - 65s 2s/step - loss: 0.0076 - val_loss: 0.0058
Epoch 8/50
36/36 [==============================] - 65s 2s/step - loss: 0.0061 - val_loss: 0.0051
Epoch 9/50
36/36 [==============================] - 65s 2s/step - loss: 0.0056 - val_loss: 0.0058
Epoch 10/50
36/36 [==============================] - 65s 2s/step - loss: 0

In [ ]:
def test(_y,_x):
  predt = model.predict(xt)
  predt[:,_y,_x] = 0
  pred = np.zeros(predt.shape)
  for t in range(predt.shape[0]):
    pred[t] = predt[t].T

  calculate_errors(pred, internet_origin, internet_min, internet_max, log10=True)